<a href="https://colab.research.google.com/github/guerrac2001/Colab-Notebooks/blob/main/DataYFinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install yfinance


In [3]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import (Dataset, TensorDataset,
 DataLoader, Subset)
import numpy as np
import os
import random

In [ ]:
import numpy as np
import yfinance as yahooFinance
import datetime
starDate = datetime.datetime (2014, 4, 1)
endDate = datetime.datetime.now()
dataM = yahooFinance.download("GOOG",start = starDate, end=endDate, interval = "1mo", auto_adjust = True)
print (dataM)

In [ ]:
PreciosCierre = dataM['Close'].values
print(PreciosCierre)

In [6]:
%cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [7]:
import datacnn

In [9]:
from datacnn import crear_data_entrada

x, y= crear_data_entrada (PreciosCierre)
print (x)
print (y)

[[ 525.21801758]
 [ 558.35705566]
 [ 573.70489502]
 [ 570.03497314]
 [ 570.03497314]
 [ 575.7791748 ]
 [ 557.54925537]
 [ 540.34649658]
 [ 524.95874023]
 [ 533.05651855]
 [ 556.87109375]
 [ 546.49957275]
 [ 537.34002686]
 [          nan]
 [ 532.10998535]
 [ 520.51000977]
 [ 625.60998535]
 [ 618.25      ]
 [ 608.41998291]
 [ 710.80999756]
 [ 742.59997559]
 [ 758.88000488]
 [ 742.95001221]
 [ 697.77001953]
 [ 744.95001221]
 [ 693.01000977]
 [ 735.7199707 ]
 [ 692.09997559]
 [ 768.78997803]
 [ 767.04998779]
 [ 777.28997803]
 [ 784.53997803]
 [ 758.03997803]
 [ 771.82000732]
 [ 796.78997803]
 [ 823.21002197]
 [ 829.55999756]
 [ 905.96002197]
 [ 964.85998535]
 [ 908.72998047]
 [ 930.5       ]
 [ 939.33001709]
 [ 959.10998535]
 [1016.64001465]
 [1021.40997314]
 [1046.40002441]
 [1169.93994141]
 [1104.72998047]
 [1031.79003906]
 [1017.33001709]
 [1084.98999023]
 [1115.65002441]
 [1217.26000977]
 [1218.18994141]
 [1193.4699707 ]
 [1076.77001953]
 [1094.43005371]
 [1035.60998535]
 [1116.3699951

In [ ]:
x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float().unsqueeze(dim=1)